In [51]:
try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

# import python libraries

# initialize
from pyspark.sql import SparkSession

from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as F
from pyspark.sql import Row

# from pyspark.ml.feature import HashingTF, IDF, RegexTokenizer, StringIndexer, NGram
# from pyspark.ml.classification import LogisticRegression
# from pyspark.ml import Pipeline
# from pyspark.mllib.evaluation import MulticlassMetrics

spark = SparkSession.builder.master("local[*]").getOrCreate()


In [42]:
df = spark.read.option("sep", "\t").option("header", "true").csv("../Downloads/2005_-_2015_Graduation_Outcomes.tsv")

In [10]:
# df.printSchema()
df.select('Demographic').show(truncate=False)
df.createOrReplaceTempView("dfnew")
df.toPandas().head()
# spark.sql("select distinct Demographic from dfnew").show(truncate=False)

+------------------------+
|Demographic             |
+------------------------+
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
|English Language Learner|
+------------------------+
only showing top 20 rows



Cohort Year Cohort Category               Demographic # Total Cohort  \
0        2001    4 Year  June  English Language Learner          10540   
1        2001    5 Year  June  English Language Learner          10540   
2        2001          6 Year  English Language Learner          10540   
3        2002    4 Year  June  English Language Learner           7454   
4        2002    5 Year  June  English Language Learner           7454   

  # Total Grads % of cohort Total Grads # of cohort Total Grads  \
0          2791                    26.5                     992   
1          3920                    37.2                    1233   
2          4296                    40.8                    1291   
3          1691                    22.7                     639   
4          2354                    31.6                     794   

  % of cohort Total Regents % of grads  Total Regents  \
0                         9                      35.5   
1                      11.7                      31.5   
2                      12.2                      30.1   
3                       8.6                      37.8   
4                      10.7                      33.7   

  # of grads  Total Regents  ... # of grads  Advanced Regents  \
0                       315  ...                          677   
1                       371  ...                          862   
2                       380  ...                          911   
3                       132  ...                          507   
4                       176  ...                          618   

  % of cohort  Regents w/o Advanced % of grads  Regents w/o Advanced  \
0                               6.4                             24.3   
1                               8.2                               22   
2                               8.6                             21.2   
3                               6.8                               30   
4                               8.3                             26.3   

  # of grads  Regents w/o Advanced % of cohort Local % of grads Local  \
0                             1803              17.1             64.6   
1                             2701              25.6             68.9   
2                             3028              28.7             70.5   
3                             1062              14.2             62.8   
4                             1578              21.2               67   

  # Still Enrolled % of cohort Still Enrolled # Dropped Out  \
0             3895                         37          3220   
1             1516                       14.4          4193   
2              655                        6.2          4598   
3             3062                       41.1          2330   
4             1457                       19.5          3041   

  % of cohort Dropped Out  
0                    30.6  
1                    39.8  
2                    43.6  
3                    31.3  
4                    40.8  

[5 rows x 22 columns]

In [80]:
df = spark.read.option("sep", "\t").option("header", "true").csv("../Downloads/311_Service_Requests_for_2006.tsv")

In [81]:
df.columns
df.select('Unique Key', 'Created Date', 'Closed Date', 'Incident Zip', 'Borough').show(truncate=False)

+----------+----------------------+----------------------+------------+-----------+
|Unique Key|Created Date          |Closed Date           |Incident Zip|Borough    |
+----------+----------------------+----------------------+------------+-----------+
|22665     |08/23/2006 12:00:00 AM|08/23/2006 12:00:00 AM|null        |Unspecified|
|218       |09/18/2006 12:00:00 AM|09/18/2006 12:00:00 AM|null        |Unspecified|
|220       |09/18/2006 12:00:00 AM|09/26/2006 12:00:00 AM|11692       |QUEENS     |
|2355      |09/18/2006 12:00:00 AM|09/18/2006 12:00:00 AM|null        |Unspecified|
|8638      |08/29/2006 12:00:00 AM|09/14/2006 12:00:00 AM|30091-6700  |Unspecified|
|13582     |08/23/2006 12:00:00 AM|08/23/2006 12:00:00 AM|null        |BROOKLYN   |
|13383     |08/23/2006 12:00:00 AM|08/23/2006 12:00:00 AM|null        |Unspecified|
|13384     |08/23/2006 12:00:00 AM|08/23/2006 12:00:00 AM|null        |Unspecified|
|29831     |07/27/2006 12:00:00 AM|07/27/2006 12:00:00 AM|null        |Unspe

In [54]:
import numpy as np
import pandas as pd

In [46]:
df.count()

1789359

In [61]:
def correct_zip(zip_code):
    try:
        zip_code = int(float(zip_code))
    except:
        try:
            zip_code = int(float(zip_code.split('-')[0]))
        except:
            return None
    if zip_code < 10000 or zip_code > 19999:
        return None
    else:
        return str(zip_code)

def remove_borough(b):
    if b=="Unspecified":
        return None
    else:
        return b
remove_borough = F.udf(remove_borough, StringType())
correct_zip = F.udf(correct_zip, StringType())

In [65]:
df.columns
df.select("Latitude", "Longitude").show()
newdf = df.filter(df.Latitude.isNotNull())
newdf = newdf.filter(df.Longitude.isNotNull())
newdf = newdf.filter(F.col("Closed Date").isNotNull())
newdf.select("Latitude", "Longitude").show()
newdf.count()

+--------+---------+
|Latitude|Longitude|
+--------+---------+
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
|    null|     null|
+--------+---------+
only showing top 20 rows

+---------------+----------------+
|       Latitude|       Longitude|
+---------------+----------------+
|40.727090279701|-73.976638730132|
|40.605520957995|-74.002117592825|
| 40.67560770823|-73.728742151361|
|40.636454776697|-74.000526040593|
|40.672619197264| -73.95435384966|
|40.710023243413|-73.916977799538|
|40.821556094196|-73.896887434225|
|40.727090279701|-73.976638730132|
|40.763048649245|-73.902342276776|
|40.636454776697|-74.000526040593|
|40.63645477669

961946

In [82]:
# df = df.na.fill("null")
df.createOrReplaceTempView("df")
spark.sql("select distinct `Incident Zip`from df").rdd.take(50)
df.select('Incident Zip', 'Borough').show()
df = df.withColumn("zip", correct_zip(F.col("Incident Zip")))
df.select("zip").show()
df = df.withColumn('borough', remove_borough(F.col("Borough")))
df = df.filter(df.borough.isNotNull())
df = df.filter(df.Latitude.isNotNull())
df = df.filter(df.Longitude.isNotNull())
df = df.filter(F.col("Closed Date").isNotNull())
df.select('borough').show()
df.count()

+------------+-----------+
|Incident Zip|    Borough|
+------------+-----------+
|        null|Unspecified|
|        null|Unspecified|
|       11692|     QUEENS|
|        null|Unspecified|
|  30091-6700|Unspecified|
|        null|   BROOKLYN|
|        null|Unspecified|
|        null|Unspecified|
|        null|Unspecified|
|        null|Unspecified|
|        null|Unspecified|
|        null|  MANHATTAN|
|  10549-1009|Unspecified|
|        null|Unspecified|
|        null|Unspecified|
|        null|Unspecified|
|        null|  MANHATTAN|
|       00000|      BRONX|
|       10128|  MANHATTAN|
|  46247-0707|Unspecified|
+------------+-----------+
only showing top 20 rows

+-----+
|  zip|
+-----+
| null|
| null|
|11692|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
|10549|
| null|
| null|
| null|
| null|
| null|
|10128|
| null|
+-----+
only showing top 20 rows

+---------+
|  borough|
+---------+
|MANHATTAN|
| BROOKLYN|
|   QUEENS|
| BROOKLYN|
| BROOKLYN|
|   QUEENS|


961946

In [84]:
spark.sql("select distinct agency from df").show()
spark.sql("select distinct `Complaint Type` from df").show()
spark.sql("Select distinct City from df").show(df.count())

+----------+
|    agency|
+----------+
|       DOT|
|       DCA|
|       HPD|
|       DPR|
|       TLC|
|     3-1-1|
|      NYPD|
|       DOB|
|       DEP|
|       DOE|
|     DOHMH|
|      DSNY|
|NYCSERVICE|
|     DOITT|
+----------+

+--------------------+
|      Complaint Type|
+--------------------+
|Traffic Signal Co...|
| Cranes and Derricks|
|            ELECTRIC|
|Broken Parking Meter|
|        Window Guard|
|   Broken Muni Meter|
|   Highway Condition|
|    Street Condition|
| Hazardous Materials|
|             Vending|
|     PAINT - PLASTER|
|DOE Complaint or ...|
|GENERAL CONSTRUCTION|
|Municipal Parking...|
|     Taxi Compliment|
|    Animal in a Park|
|             Smoking|
|Noise - House of ...|
|Sanitation Condition|
|           APPLIANCE|
+--------------------+
only showing top 20 rows

+--------------------+
|                City|
+--------------------+
|       VALLEY STREAM|
|              GILROY|
|              ATLANA|
|          CEDARKNOOS|
|            ELMSFORT|
|  

In [91]:
df.count()
df.select('City','borough').show(1000)

+-------------------+-------------+
|               City|      borough|
+-------------------+-------------+
|           NEW YORK|    MANHATTAN|
|           BROOKLYN|     BROOKLYN|
|           ROSEDALE|       QUEENS|
|           BROOKLYN|     BROOKLYN|
|           BROOKLYN|     BROOKLYN|
|          RIDGEWOOD|       QUEENS|
|              BRONX|        BRONX|
|           NEW YORK|    MANHATTAN|
|            ASTORIA|       QUEENS|
|           BROOKLYN|     BROOKLYN|
|           BROOKLYN|     BROOKLYN|
|           BROOKLYN|     BROOKLYN|
|           BROOKLYN|     BROOKLYN|
|           BROOKLYN|     BROOKLYN|
|           BROOKLYN|     BROOKLYN|
|          GLEN OAKS|       QUEENS|
|          GLEN OAKS|       QUEENS|
|           NEW YORK|    MANHATTAN|
|          RIDGEWOOD|       QUEENS|
|          RIDGEWOOD|       QUEENS|
|           NEW YORK|    MANHATTAN|
|           NEW YORK|    MANHATTAN|
|           NEW YORK|    MANHATTAN|
|           NEW YORK|    MANHATTAN|
|           NEW YORK|    MAN

In [97]:
df.select('City').distinct().show(df.count())
df.select('Status').distinct().show(df.count())
df.select('Borough').distinct().show(df.count())
df.groupBy('Agency').count().select('Agency', F.col("count").alias("ctr")).show()

+-------------------+
|               City|
+-------------------+
|             CORONA|
|            BAYSIDE|
|       FAR ROCKAWAY|
|        KEW GARDENS|
|        FLORAL PARK|
|               null|
|       BREEZY POINT|
|         WHITESTONE|
|    CAMBRIA HEIGHTS|
|          REGO PARK|
|   SOUTH OZONE PARK|
|        LITTLE NECK|
|    OAKLAND GARDENS|
|             QUEENS|
|      ROCKAWAY PARK|
|           BROOKLYN|
|           ELMHURST|
|      EAST ELMHURST|
|           FLUSHING|
|    JACKSON HEIGHTS|
|          WOODHAVEN|
|            MASPETH|
|     MIDDLE VILLAGE|
|   LONG ISLAND CITY|
|          BELLEROSE|
|         OZONE PARK|
|              BRONX|
|SOUTH RICHMOND HILL|
|      FRESH MEADOWS|
|      NEW HYDE PARK|
|                 NY|
|            ASTORIA|
|SPRINGFIELD GARDENS|
|            ARVERNE|
|           NEW YORK|
|       HOWARD BEACH|
|          SUNNYSIDE|
|       SAINT ALBANS|
|            JAMAICA|
|       FOREST HILLS|
|           ROSEDALE|
|      RICHMOND HILL|
|         

In [107]:
df.groupBy("Complaint Type", "Borough").count().select('Complaint Type','Borough', F.col("count").alias("ctr")).orderBy([F.col("ctr"), F.col("Borough")], ascending=False).show(df.count(), truncate=False)





+---------------------------------------+-------------+-----+
|Complaint Type                         |Borough      |ctr  |
+---------------------------------------+-------------+-----+
|Street Light Condition                 |QUEENS       |34856|
|Sewer                                  |QUEENS       |28850|
|Street Light Condition                 |BROOKLYN     |22586|
|Sewer                                  |BROOKLYN     |22221|
|Noise                                  |MANHATTAN    |21001|
|Water System                           |QUEENS       |20556|
|Blocked Driveway                       |QUEENS       |20416|
|Water System                           |BROOKLYN     |20322|
|Blocked Driveway                       |BROOKLYN     |19488|
|Traffic Signal Condition               |BROOKLYN     |18433|
|General Construction/Plumbing          |BROOKLYN     |17053|
|Building/Use                           |QUEENS       |16988|
|Noise - Street/Sidewalk                |MANHATTAN    |15668|
|Traffic